In [2]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import os

Using TensorFlow backend.


In [3]:
import keras
from keras.models import load_model

In [4]:
model = load_model('2new-weights-improvement-33-3.2389-bigger.hdf5')

In [5]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "new-weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=50, batch_size=128, callbacks=callbacks_list)

In [6]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [16]:
def get_notes():
    
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []
    n=0
    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)
        n+=1
        print(n)
        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes_new', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [15]:
len(glob.glob("midi_songs/*.mid"))

62

In [18]:
notes = get_notes()

1
Parsing midi_songs/大田後生仔.wav.mid
2
Parsing midi_songs/說散就散.wav.mid
3
Parsing midi_songs/透明.wav.mid
4
Parsing midi_songs/123我愛你.wav.mid
5
Parsing midi_songs/多想在平庸的生活擁抱你.wav.mid
6
Parsing midi_songs/MOM.wav.mid
7
Parsing midi_songs/你说.wav.mid
8
Parsing midi_songs/病變.wav.mid
9
Parsing midi_songs/醉赤壁.wav.mid
10
Parsing midi_songs/你保护世界我保护你.wav.mid
11
Parsing midi_songs/晚安.wav.mid
12
Parsing midi_songs/38度6.wav.mid
13
Parsing midi_songs/學貓叫.wav.mid
14
Parsing midi_songs/最天使.wav.mid
15
Parsing midi_songs/芒種.wav.mid
16
Parsing midi_songs/纸短情长.wav.mid
17
Parsing midi_songs/講真的.wav.mid
18
Parsing midi_songs/剛好遇見你.wav.mid
19
Parsing midi_songs/空空如也.wav.mid
20
Parsing midi_songs/疑心病.wav.mid
21
Parsing midi_songs/體面.wav.mid
22
Parsing midi_songs/野狼disco.wav.mid
23
Parsing midi_songs/綠色.wav.mid
24
Parsing midi_songs/重生.wav.mid
25
Parsing midi_songs/除了春天爱情和樱花.wav.mid
26
Parsing midi_songs/一千零一次我愛你.wav.mid
27
Parsing midi_songs/告白氣球.wav.mid
28
Parsing midi_songs/愛你.wav.mid
29
Parsing midi_songs/来不及

In [25]:
# get amount of pitch names
n_vocab = len(set(notes))

network_input, network_output = prepare_sequences(notes, n_vocab)

model = load_model('new-weights-improvement-50-3.4214-bigger.hdf5')



In [26]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6684502494543449537
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11714348434584700205
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10990990132
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 2219939124139933076
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:8a:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5280113212374629939
physical_device_desc: "device: XLA_GPU device"
]


In [27]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [28]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "2new-weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=50, batch_size=128, callbacks=callbacks_list)

In [ ]:
train(model, network_input, network_output)

Epoch 1/50
227936/227936 [==============================] - 493s 2ms/step - loss: 3.4181
Epoch 2/50
227936/227936 [==============================] - 494s 2ms/step - loss: 3.4155
Epoch 3/50
227936/227936 [==============================] - 487s 2ms/step - loss: 3.4019
Epoch 4/50
227936/227936 [==============================] - 488s 2ms/step - loss: 3.4023
Epoch 5/50
227936/227936 [==============================] - 489s 2ms/step - loss: 3.3951
Epoch 6/50
227936/227936 [==============================] - 490s 2ms/step - loss: 3.3884
Epoch 7/50
227936/227936 [==============================] - 493s 2ms/step - loss: 3.3784
Epoch 8/50
227936/227936 [==============================] - 491s 2ms/step - loss: 3.3713
Epoch 9/50
227936/227936 [==============================] - 494s 2ms/step - loss: 3.3622
Epoch 10/50
227936/227936 [==============================] - 496s 2ms/step - loss: 3.3592
Epoch 11/50
227936/227936 [==============================] - 496s 2ms/step - loss: 3.3555
Epoch 12/50
227936/

references:
* https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
* https://github.com/Skuldur/Classical-Piano-Composer
* https://keras.io/api/layers/recurrent_layers/lstm/